# Algoritmo DFP y BFGS 

In [1]:
#parte 1 de las funciones 
#algorimto de DFP

from scipy.optimize import line_search
from numpy import linalg as la
import numpy as np
import time

In [43]:
#Algoritmos lineales Cuasi Newton

class algoritmos_cuasinewton(object):
    def __init__(self):
        self.algoritmos={'DFP':self._DFP, 'BFGS':self._BFGS}
        
    def _backtracking(self,f, g, xk, dk, fk, gk, alpha0, c1=0.01, c2=0.9, max_iter=100,tolalpha=1e-14):

        alpha = 0
        beta = float('inf')
        alphai = alpha0
        it = 0
        while 1 and it < max_iter:
            if f(xk + alphai*dk) > fk + c1*alphai*np.dot(np.transpose(gk), dk):
                beta = alphai
                alphai = (alpha + beta)/2
            elif np.dot(np.transpose(g(xk + alphai*dk)), dk) < c2*np.dot(np.transpose(gk), dk):
                alpha = alphai
                if beta == float('inf'):
                    alphai = 2*alpha
                else:
                    alphai = (alpha + beta)/2
            else:
                break
            it+=1
        return alphai    
    def _DFP(self,x0,tf,grad,H0,tolx=1e-9,tolf=1e-9,tolg=1e-4,maxitr=10000):
        iterations=0
        tolerx=1e3
        tolerf=1e3
        xk=x0
        Hk=H0
        gk=grad(xk)
        while la.norm(gk)>tolg and iterations<maxitr:
            dk=-np.dot(Hk,gk)
            fk=tf(xk)
            step=self._backtracking(tf,grad,xk,dk,fk,gk,10.0)
            new=xk+step*dk
            gnew=grad(new)
            yk=gnew-gk
            sk=new-xk
            #La parte de las tolerancias afetan el rendimiento del algoritmo
            #tolerx=abs(la.norm(sk))/max(1,abs(la.norm(xk))) #tolerancia de las xk
            #if tolerx<tolx:
            #    print("Tolerancia de las x")
            #    break
            #tolerf=abs(tf(new)-tf(xk))/max(1,abs(tf(xk))) #tolerancia de las f 
            #if tolerf<tolf:
            #    print("Tolerancia de f")
            #    break
            Hk=Hk-np.dot(Hk,np.dot(yk,np.dot(yk.T,Hk)))*(1/np.dot(yk.T,np.dot(Hk,yk)))+np.dot(sk,sk.T)*(1/np.dot(yk.T,sk))
            gk=gnew
            xk=new
            iterations+=1
        return np.array([xk,iterations,la.norm(grad(xk))])

    def _BFGS (self,x0,tf,grad,H0,tolx=1e-9,tolf=1e-9,tolg=1e-4,maxitr=10000):
        iterations=0
        tolerx=1e3
        tolerf=1e3
        xk=x0
        Hk=H0
        gk=grad(xk)
        while la.norm(grad(xk))>tolg and iterations<maxitr:
            dk=-np.dot(Hk,gk)
            dk/=la.norm(dk)
            fk=tf(xk)
            #dk=np.reshape(dk,(dk.shape[0],1))
            step=self._backtracking(tf,grad,xk,dk,fk,gk,100.0)
            new=xk+step*dk
            gnew=grad(new)
            yk=gnew-gk
            sk=new-xk
            rk=1/max(1,np.dot(yk.T,sk))
            #Vamos a quitar las tolerancias porque se afecta el rendimiento del algoritmo
            #tolerx=abs(la.norm(sk))/max(1,abs(la.norm(xk))) #tolerancia de las xk
            #if tolerx<tolx:
            #    print("Tolerancia de las x")
            #    break
            #tolerf=abs(tf(new)-tf(xk))/max(1,abs(tf(xk))) #tolerancia de las f 
            #if tolerf<tolf:
            #    print("Tolerancia de las f")
            #    break
            Hk=np.dot((np.eye(Hk.shape[0],Hk.shape[1])-rk*np.dot(sk,yk.T)),np.dot(Hk,(np.eye(Hk.shape[0],Hk.shape[1])-rk*np.dot(yk,sk.T)))) + rk*np.dot(sk,sk.T)
            xk=new
            gk=gnew
            iterations+=1
        return np.array([xk,iterations,la.norm(grad(xk))])
    def evaluate(self,f,grad,dim,algoritmo='BFGS'):
        if algoritmo not in self.algoritmos.keys():
            return None
        else:
            H0=np.eye(dim)
            iteraciones=0
            norma=0
            tiempo=0
            np.random.seed(0)
            for i in range(30):
                test=np.random.rand(dim,1)
                start=time.clock()
                res=self.algoritmos[algoritmo](test,f,grad,H0)
                end=time.clock()
                norma+=res[2]
                tiempo+=(end-start)
                iteraciones+=res[1]
                print("Evaluación numero: ",i+1)
            return np.array([norma/30,tiempo/30,round(iteraciones/30)])

# Funciones de Prueba 

### Funcion Rosenbrock para n=100

In [44]:
def rosen(x): #La función de rosembruck en general
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

def r_der(x):
        xm = x[1:-1]
        xm_m1 = x[:-2]
        xm_p1 = x[2:]
        der =np.zeros(x.shape)
        der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
        der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
        der[-1] = 200*(x[-1]-x[-2]**2)
        return der

def rosen_hess(x):
        H = np.diag(-400*x[:-1],1) - np.diag(400*x[:-1],-1)
        diagonal = np.zeros(len(x))
        diagonal[0] = 1200*x[0]-400*x[1]+2
        diagonal[-1] = 200
        diagonal[1:-1] = 202 + 1200*x[1:-1]**2 - 400*x[2:]
        H = H + np.diag(diagonal)
        return H

In [48]:
cn=algoritmos_cuasinewton()

np.random.seed(0)
test=np.random.rand(100,1)
H0=np.eye(test.shape[0])
print(test.shape,H0.shape)
res=cn.algoritmos['BFGS'](test,rosen,r_der,H0)

(100, 1) (100, 100)


In [49]:
print(res[2])

8.139220964052469e-05


In [50]:
#Vector de prueba del algoritmo

cn=algoritmos_cuasinewton()


res=cn.evaluate(rosen,r_der,100)

/home/danielvaal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
/home/danielvaal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:102: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


Evaluación numero:  1
Evaluación numero:  2
Evaluación numero:  3
Evaluación numero:  4
Evaluación numero:  5
Evaluación numero:  6
Evaluación numero:  7
Evaluación numero:  8
Evaluación numero:  9
Evaluación numero:  10
Evaluación numero:  11
Evaluación numero:  12
Evaluación numero:  13
Evaluación numero:  14
Evaluación numero:  15
Evaluación numero:  16
Evaluación numero:  17
Evaluación numero:  18
Evaluación numero:  19
Evaluación numero:  20
Evaluación numero:  21
Evaluación numero:  22
Evaluación numero:  23
Evaluación numero:  24
Evaluación numero:  25
Evaluación numero:  26
Evaluación numero:  27
Evaluación numero:  28
Evaluación numero:  29
Evaluación numero:  30


In [51]:
print("Algoritmo BFGS")
print(res)

Algoritmo BFGS
[4.23259630e-01 1.87405304e+01 6.22400000e+03]


In [52]:
res=cn.evaluate(rosen,r_der,100,algoritmo='DFP')

/home/danielvaal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
/home/danielvaal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:102: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


Evaluación numero:  1
Evaluación numero:  2
Evaluación numero:  3
Evaluación numero:  4
Evaluación numero:  5
Evaluación numero:  6
Evaluación numero:  7
Evaluación numero:  8
Evaluación numero:  9
Evaluación numero:  10
Evaluación numero:  11
Evaluación numero:  12
Evaluación numero:  13
Evaluación numero:  14
Evaluación numero:  15
Evaluación numero:  16
Evaluación numero:  17
Evaluación numero:  18
Evaluación numero:  19
Evaluación numero:  20
Evaluación numero:  21
Evaluación numero:  22
Evaluación numero:  23
Evaluación numero:  24
Evaluación numero:  25
Evaluación numero:  26
Evaluación numero:  27
Evaluación numero:  28
Evaluación numero:  29
Evaluación numero:  30


In [53]:
print("Algoritmo DFP")
print(res)

Algoritmo DFP
[5.20510097e-05 2.97823033e+00 3.21400000e+03]


In [54]:
def wood(x): #Función general, target function
    return 100*((x[1]-x[0]**2)**2)+(1-x[0])**2+(1-x[2])**2+90*((x[3]-x[2]**2)**2)+10.1*((x[1]-1)**2+(x[3]-1)**2)+19.8*(x[1]-1)*(x[3]-1)

def wood_derivate(x):
    g=np.zeros((len(x),1))
    g[0]=2.0*(200.0*x[0]**3-200*x[0]*x[1]+x[0]-1)
    g[1]=-200*x[0]**2+220.2*x[1]+19.8*x[3]-40
    g[2]=2*(180*x[2]**3-180*x[3]*x[2]+x[2]-1)
    g[3]=19.8*x[1]-180*x[2]**2+200.2*x[3]-40
    return g

def wood_hessian(x):
    hes=np.zeros((len(x),len(x)))
    hes[0][0]=400*(x[0]**2-x[1])+800*x[0]**2+2
    hes[0][1]=-400*x[0]
    hes[1][0]=400*x[0]
    hes[1][1]=220.2
    hes[1][3]=19.8
    hes[2][2]=2+720*x[2]**2+360*(x[2]**2-x[3])
    hes[2][3]=-360*x[2]
    hes[3][1]=19.8
    hes[3][2]=-360*x[2]
    hes[3][3]=200.2
    return hes

In [55]:
np.random.seed(0)
test=np.random.rand(4,1)
H0=np.eye(test.shape[0])
print(test.shape,H0.shape)
res=cn.algoritmos['DFP'](test,wood,wood_derivate,H0)

(4, 1) (4, 4)


In [56]:
print(res)

[array([[1.        ],
       [0.99999994],
       [1.00000004],
       [0.99999995]])
 27 5.7886758552209355e-05]


In [59]:
print("WOOD DFP")
res=cn.evaluate(wood,wood_derivate,4,algoritmo='DFP')
print(res)

WOOD DFP
Evaluación numero:  1
Evaluación numero:  2
Evaluación numero:  3
Evaluación numero:  4
Evaluación numero:  5
Evaluación numero:  6
Evaluación numero:  7
Evaluación numero:  8
Evaluación numero:  9
Evaluación numero:  10
Evaluación numero:  11
Evaluación numero:  12
Evaluación numero:  13


/home/danielvaal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
/home/danielvaal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:102: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


Evaluación numero:  14
Evaluación numero:  15
Evaluación numero:  16
Evaluación numero:  17
Evaluación numero:  18
Evaluación numero:  19
Evaluación numero:  20
Evaluación numero:  21
Evaluación numero:  22
Evaluación numero:  23
Evaluación numero:  24
Evaluación numero:  25
Evaluación numero:  26
Evaluación numero:  27
Evaluación numero:  28
Evaluación numero:  29
Evaluación numero:  30
[5.89442622e-05 1.53116333e-02 5.80000000e+01]


In [60]:
print("WOOD BFGS")
res=cn.evaluate(wood,wood_derivate,4)
print(res)

WOOD BFGS
Evaluación numero:  1


/home/danielvaal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
/home/danielvaal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:102: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


Evaluación numero:  2
Evaluación numero:  3
Evaluación numero:  4
Evaluación numero:  5
Evaluación numero:  6
Evaluación numero:  7
Evaluación numero:  8
Evaluación numero:  9
Evaluación numero:  10
Evaluación numero:  11
Evaluación numero:  12
Evaluación numero:  13
Evaluación numero:  14
Evaluación numero:  15
Evaluación numero:  16
Evaluación numero:  17
Evaluación numero:  18
Evaluación numero:  19
Evaluación numero:  20
Evaluación numero:  21
Evaluación numero:  22
Evaluación numero:  23
Evaluación numero:  24
Evaluación numero:  25
Evaluación numero:  26
Evaluación numero:  27
Evaluación numero:  28
Evaluación numero:  29
Evaluación numero:  30
[8.30001995e-05 1.26689867e-01 1.62000000e+02]
